In [1]:
import pandas as pd
import json
import sys
import os
import ast


sys.path.append('../../../')

#import aizynthfinder.chem as chem
import aizynthfinder.analysis

from aizynthfinder.mcts.node import Node
from aizynthfinder.analysis import ReactionTree
from aizynthfinder.mcts.state import State
from aizynthfinder.chem import TreeMolecule
from aizynthfinder.context.collection import ContextCollection
from aizynthfinder.context.stock import StockException

from Maranga.utils.parse_multismiles import findkeys

import matplotlib.pyplot as plt

In [2]:
# read in template file
f = '/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/aizynthfinder/data/uspto_templates.hdf5'
data = pd.read_hdf(f, 'table')

In [6]:
#convert 'clasification column to list'
classes = data['classification'].tolist()
unique_class = list(set(classes))


In [13]:
import random

rand27 = random.sample(unique_class, 27)
print(rand27)

rand10 = random.sample(unique_class, 10)
print(rand10)

['Other acylation', 'Nitrile to acid', 'Other C-C bond formation', 'Other Pd-catalyzed reactions (Negishi,Kumada,etc.)', 'N-arylation with Ar-X', 'Alcohols to acids', 'Cyano or imine to amine', 'Salt formation', 'S-containing heterocycle formation', 'Unassigned', 'Ketone to alcohol', 'ROH deprotections', 'RCO2H deprotections', 'N-acylation to urea', 'O-sulfonylation', 'Heck reaction', 'Heteroaryl N-alkylation', 'Dehydration', 'N-sulfonylation', 'N-containing heterocycle formation', 'Other oxidations', 'Other deprotections', 'Alkene oxidative cleavage', 'Alcohol to halide', 'Suzuki coupling', 'Halogenation', 'Alcohols to aldehydes']
['N-containing heterocycle formation', 'RCO2H protections', 'Heteroaryl N-alkylation', 'N-arylation with Ar-X', 'Alkene oxidation', 'O-acylation to ester', 'Nitrile to acid', 'N-substitution with alkyl-X', 'Salt formation', 'Unassigned']
